In [1]:
import sys
import os

# Add parent directory to system path
sys.path.append(os.path.abspath(".."))
import connect_db

from importlib import reload

reload(connect_db)
ConnectDB = connect_db.ConnectDB

In [5]:
import json

# Path to the JSON file
file_path = '../temp.json'

# Open the JSON file and load its contents
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Print the loaded data
print(len(data))


4


In [2]:
db = ConnectDB(db_path='../app_storage/metadata/sqlite-poc.db')
db.connection.cursor().execute("SELECT * FROM pdf_metadata").fetchall()

OperationalError: no such table: pdf_metadata

In [12]:
import sqlite3
import pandas as pd
import sqlite_vec



sqlite_connection = sqlite3.connect('../app_storage/metadata/sqlite-poc.db')
sqlite_connection.enable_load_extension(True)
sqlite_vec.load(sqlite_connection)
sqlite_connection.enable_load_extension(False)


pd.read_sql_query("SELECT * FROM pdf_metadata;", sqlite_connection).head()


,id,file_name,title,author,subject,keywords,creation_date
0,1,None,,,,,2024-04-10
1,2,None,,,,,2024-04-10
2,3,None,,,,,2024-04-10
3,4,None,Language Models are Unsupervised Multitask Lea...,"Alec Radford*, Jeffrey Wu*, Rewon Child, David...",Proceedings of the International Conference on...,"Machine Learning, ICML",2019-02-14
4,5,None,,,,,2020-07-24


In [27]:
chunk_id = 4
cursor = sqlite_connection.cursor()

cursor.execute("""
        SELECT text FROM chunks WHERE chunk_id = ?
    """, (chunk_id,))
cursor.close()
sqlite_connection.close()

In [21]:
test = {'document_name': 'Attention is all you need',
  'metadata': {'title': '',
   'author': '',
   'subject': '',
   'keywords': '',
   'creation_date': '2024-04-10'},
  'sections': [{'section': 'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.',
    'text': 'Attention Is All You Need arXiv:1706.03762v7  [cs.CL]  2 Aug 2023 Noam Shazeer∗\nGoogle Brain\nnoam@google.com Ashish Vaswani∗\nGoogle Brain\navaswani@google.com Łukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com Aidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu Llion Jones∗\nGoogle Research\nllion@google.com Illia Polosukhin∗‡\nillia.polosukhin@gmail.com',
    'pages': ['1'],
    'document': 'Attention is all you need',
    'word_count': 40,
    'hash': '590cac8437f2f1fe'},
   {'section': '‡Work performed while at Google Research.',
    'text': '‡Work performed while at Google Research.',
    'pages': ['1'],
    'document': 'Attention is all you need',
    'word_count': 6,
    'hash': '00cbf2762187a65e'}]}
test.get("document_name")

'Attention is all you need'